Ideas for performance improvement:
- Static Tipying: NO!
- Numba: NoPyton, Parallelize, TypeSpecification, Nogil?
- For Random Numbers two function (Cuda vs NoCuda)

# PRODUCTION READY FUNCTION => Simulator_noGPU

In [1]:
# Load the minimum required library to run the functions
from numba import jit
from numpy import zeros, arange, uint8, int32, float32, sqrt, uint32, ones, int64, float64
from numpy.random import randn, uniform
from timeit import timeit
import time
import matplotlib.pyplot as plt
import numpy as np
from numba import cuda

from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32, xoroshiro128p_normal_float32

In [ ]:
print(f"Hello {1001010.09302131:.2f}")
print("Your Integration Time (dt) is %.2E seconds, %.2f" % (dt, 0.0001))

In [19]:
def CheckParameters(dt, DeltaT, TotalT, theta):
    time_steps_amount = int64(TotalT/dt) # Number of steps
    sampled_point_amount = int64(TotalT/DeltaT) # Number of sampled points
    sampling_delta_time_steps = int64(DeltaT/dt) # Number of steps between samples
    n_sim = theta[0].shape[0]
    # Aggiugnere controllo sul TotalT effettivo a fine simulazione
    # Aggiungere controllo sul sampling_delta_time_steps per sanity check
    # Controllare che sampled_point_amount*sampling_delta_time_steps = time_steps_amount
    
    print(f"Your Integration Time (dt) is {dt:.2E} seconds")
    print(f"Your Sampling Time (Delta) is {DeltaT:.2E} seconds, corresponding to a {1/DeltaT:.2f}Hz sampling frequency")
    print(f"Your Total Simulation Time is {TotalT:.2E} seconds")
    print(f"Your Number of Simulated Trajectories is {n_sim:.2E}")
    print(f"The amount of total time steps is {time_steps_amount:.2E}")
    print(f"The amount of sampled points is {sampled_point_amount:.2E}")
    print(f"The gap between two sampled points is {sampling_delta_time_steps:.1E} time steps")
    passed_sanity_checks = True
    print("---- SANITY CHECKS ----")
    if TotalT != DeltaT*sampled_point_amount:
        print(f"WARNING: TotalT is {TotalT}s, but DeltaT*sampled_point_amount is {DeltaT*sampled_point_amount}s")
        passed_sanity_checks = False
    if sampled_point_amount*sampling_delta_time_steps != time_steps_amount:
        print(f"WARNING: sampled_point_amount*sampling_delta_time_steps is {sampled_point_amount*sampling_delta_time_steps}, but time_steps_amount is {time_steps_amount}")
        passed_sanity_checks = False
    if time_steps_amount*dt != TotalT:
        print(f"WARNING: time_steps_amount*dt is {time_steps_amount*dt}, but TotalT is {TotalT}")
        passed_sanity_checks = False
    if dt*sampling_delta_time_steps != DeltaT:
        print(f"WARNING: dt*sampling_delta_time_steps is {dt*sampling_delta_time_steps}, but DeltaT is {DeltaT}")
        passed_sanity_checks = False
    if len(set([x.shape for x in theta])) != 1:
        raise Exception("Parameters dimension are not all equal. Detected number of different parameters: ", n_sim)
    if passed_sanity_checks:
        print("All checks passed")


In [10]:
A = [np.ones((10,10)),np.ones((5,10)),np.ones((1,10))]

b = [x.shape for x in A]
len(set(b))

3

In [14]:
@jit(nopython = True)
def Simulator_noGPU(dt, DeltaT, TotalT, theta, i_state = None, debug = False):
    
    
    time_steps_amount = int64(TotalT/dt) # Number of steps
    sampled_point_amount = int64(TotalT/DeltaT) # Number of sampled points
    sampling_delta_time_steps = int64(DeltaT/dt) # Number of steps between samples
    
    n_sim = theta[0].shape[0]
    
    # Unpack Parameters
    mu_x = theta[0]
    mu_y = theta[1]
    k_x = theta[2]
    k_y = theta[3]
    k_int = theta[4]
    tau = theta[5]
    eps = theta[6]
    D_x = theta[7]
    D_y = theta[8]
    
    
    if len(set([x.shape for x in theta])) != 1:
        raise Exception("Parameters dimension are not all equal. Detected number of different parameters: ", n_sim)
    
    
    # Handle initial state
    if i_state is None:
        x = zeros((n_sim,1), dtype = float32)
        y = zeros((n_sim,1), dtype = float32)
        f = zeros((n_sim,1), dtype = float32)
    else:
        if x.shape == (n_sim,1) or y.shape == (n_sim,1) or f.shape == (n_sim,1):
            raise Exception("Initial state has wrong shape, each should be (n_sim,1)")
        x, y, f = i_state
    
    # Initialize x_trace array
    
    x_trace = zeros((n_sim, sampled_point_amount), dtype = float32)
    
    sampling_counter = int64(1)
    
    # POSSIBLE OPTIM: You could overwrite the last used row of the trace to save memory and not create a proxy array
    
    # TRADEOFF: Memory vs Speed. We can generate the numbers here, or before. Maybe the time is the same... Should test
    
    # CHECK: Benchmark the version with the explicit dx, dy, df and the one with the x, y, f arrays with the calculation in the assigment
    
    for t in arange(time_steps_amount - 1):
        x[:,] = x[:,] + mu_x*(- k_x * x[:,] + k_int*y[:,])*dt                +          sqrt(2*mu_x*D_x*dt)     *randn(n_sim,1)
        y[:,] = y[:,] + mu_y*(-k_y*y[:,] + k_int*x[:,] + f[:,])*dt        +          sqrt(2*mu_y*D_y*dt)     *randn(n_sim,1)
        f[:,] = f[:,] + -(f[:,]/tau)*dt                                   +          sqrt(2*eps**2*dt/tau)   *randn(n_sim,1)

        sampling_counter = sampling_counter + 1
        if sampling_counter == sampling_delta_time_steps:
            x_trace[:, int(t/sampling_delta_time_steps)] = x[:,0]
            sampling_counter = int64(1)
    
    return x_trace, (x, y, f) # Check if this is right

In [ ]:
@cuda.jit(nopython = True)
def Simulator_GPU(dt, DeltaT, TotalT, n_sim, theta, rng_states, i_state = None):
    thread_id = cuda.grid(1)
    time_steps_amount = uint32(TotalT/dt) # Number of steps
    sampled_point_amount = uint32(TotalT/DeltaT) # Number of sampled points
    sampling_delta_time_steps = uint32(DeltaT/dt) # Number of steps between samples
    
    # Unpack Parameters

    mu_x = theta[0]
    mu_y = theta[1]
    k_x = theta[2]
    k_y = theta[3]
    k_int = theta[4]
    tau = theta[5]
    eps = theta[6]
    D_x = theta[7]
    D_y = theta[8]
    
    # Handle initial state
    if i_state is None:
        x = zeros((n_sim,1), dtype = float32)
        y = zeros((n_sim,1), dtype = float32)
        f = zeros((n_sim,1), dtype = float32)
    else:
        if x.shape == (n_sim,1) or y.shape == (n_sim,1) or f.shape == (n_sim,1):
            raise Exception("Initial state has wrong shape, each should be (n_sim,1)")
        x, y, f = i_state
    
    # Initialize x_trace array
    x_trace = zeros((n_sim, sampled_point_amount), dtype = float32)
    
    sampling_counter = uint8(1)
    # POSSIBLE OPTIM: You could overwrite the last used row of the trace to save memory and not create a proxy array
    
    # TRADEOFF: Memory vs Speed. We can generate the numbers here, or before. Maybe the time is the same... Should test
    
    # CHECK: Benchmark the version with the explicit dx, dy, df and the one with the x, y, f arrays with the calculation in the assigment
    
    
    for t in arange(time_steps_amount - 1):
        for i in range(n_sim):
            x[i,] = x[i,] + mu_x*(-k_x*x[i,] + k_int*y[i,])*dt                +          sqrt(2*mu_x*D_x*dt)     *xoroshiro128p_normal_float32(rng_states, thread_id)
            y[i,] = y[i,] + mu_y*(-k_y*y[i,] + k_int*x[i,] + f[:,])*dt        +          sqrt(2*mu_y*D_y*dt)     *xoroshiro128p_normal_float32(rng_states, thread_id)
            f[i,] = f[i,] + -(f[i,]/tau)*dt                                   +          sqrt(2*eps**2*dt/tau)   *xoroshiro128p_normal_float32(rng_states, thread_id)
        sampling_counter = sampling_counter + 1
        if sampling_counter == sampling_delta_time_steps:
            x_trace[:, int(t/sampling_delta_time_steps)] = x[:,0]
            sampling_counter = uint8(1)
    
    
    return x_trace, (x, y, f)

In [ ]:
@jit(nopython = True)
def Simulator_noGPU_preallocating(dt, DeltaT, TotalT, n_sim, theta, x_trace, i_state = None):
    time_steps_amount = uint32(TotalT/dt) # Number of steps
    sampled_point_amount = uint32(TotalT/DeltaT) # Number of sampled points
    sampling_delta_time_steps = uint32(DeltaT/dt) # Number of steps between samples
    
    # Unpack Parameters

    mu_x, mu_y, k_x, k_y, k_int, tau, eps, D_x, D_y = theta
    
    # Handle initial state
    if i_state is None:
        x = zeros((n_sim,1), dtype = float32)
        y = zeros((n_sim,1), dtype = float32)
        f = zeros((n_sim,1), dtype = float32)
    else:
        if x.shape == (n_sim,1) or y.shape == (n_sim,1) or f.shape == (n_sim,1):
            raise Exception("Initial state has wrong shape, each should be (n_sim,1)")
        x, y, f = i_state
    
    # Initialize x_trace array
    if x_trace is None:
        x_trace = zeros((n_sim, sampled_point_amount), dtype = float32)
    else:
        if x_trace.shape != (n_sim, sampled_point_amount):
            raise Exception("x_trace has wrong shape, should be (n_sim, sampled_point_amount)")
    
    sampling_counter = uint8(1)
    # POSSIBLE OPTIM: You could overwrite the last used row of the trace to save memory and not create a proxy array
    
    # TRADEOFF: Memory vs Speed. We can generate the numbers here, or before. Maybe the time is the same... Should test
    
    # CHECK: Benchmark the version with the explicit dx, dy, df and the one with the x, y, f arrays with the calculation in the assigment
    
    for t in arange(time_steps_amount - 1):
        x[:,] = x[:,] + mu_x*(-k_x*x[:,] + k_int*y[:,])*dt                +          sqrt(2*mu_x*D_x*dt)     *randn(n_sim,1)
        y[:,] = y[:,] + mu_y*(-k_y*y[:,] + k_int*x[:,] + f[:,])*dt        +          sqrt(2*mu_y*D_y*dt)     *randn(n_sim,1)
        f[:,] = f[:,] + -(f[:,]/tau)*dt                                   +          sqrt(2*eps**2*dt/tau)   *randn(n_sim,1)

        sampling_counter = sampling_counter + 1
        if sampling_counter == sampling_delta_time_steps:
            x_trace[:, int(t/sampling_delta_time_steps)] = x[:,0]
            sampling_counter = uint8(1)

    return (x, y, f)

In [23]:
n_sim = 10

mu_x = uniform(1.5e4, 4e4, size=(n_sim,1))
mu_y = uniform(1e4, 140e4, size=(n_sim,1))
k_x = uniform(3e-3, 16e-3, size=(n_sim,1))
k_y = uniform(1.5e-2, 30e-2, size=(n_sim,1))
k_int = uniform(1e-3, 6e-3, size=(n_sim,1))
tau = uniform(2e-2, 20e-2, size=(n_sim,1))
eps = uniform(0.5, 6, size=(n_sim,1))
D_x = uniform(5.5, 15.5, size=(n_sim,1))
D_y = uniform(1, 530, size=(n_sim,1))
sigma = uniform(4, 260, size=(n_sim,1))



dt = 1e-7
DeltaT = 0.1
TotalT = 3

theta = [mu_x, mu_y, k_x, k_y, k_int, tau, eps, D_x, D_y]

CheckParameters(dt, DeltaT, TotalT, theta)

Your Integration Time (dt) is 1.00E-07 seconds
Your Sampling Time (Delta) is 1.00E-01 seconds, corresponding to a 10.00Hz sampling frequency
Your Total Simulation Time is 3.00E+00 seconds
Your Number of Simulated Trajectories is 1.00E+01
The amount of total time steps is 3.00E+07
The amount of sampled points is 3.00E+01
The gap between two sampled points is 1.0E+06 time steps
---- SANITY CHECKS ----


In [24]:
x_trace, state = Simulator_noGPU(dt, DeltaT, TotalT, theta, debug = True)
x_trace

array([[ 6.55091333e+00, -1.45445871e+01, -3.67998314e+01,
         1.63159389e+01, -2.15399780e+01, -5.25173340e+01,
        -2.08145370e+01,  1.30508642e+01, -8.42586422e+00,
         4.16640358e+01, -7.64895821e+00, -1.64335442e+01,
        -2.19207436e-01, -3.43612442e+01, -4.09999733e+01,
         5.48598099e+00,  2.17064438e+01,  8.39467168e-01,
        -6.28337097e+01,  2.69257603e+01, -6.22230721e+01,
         2.04658356e+01, -1.54334078e+01,  2.58678493e+01,
        -7.85710955e+00,  1.70849915e+01, -1.77912102e+01,
        -1.77373295e+01,  3.11867803e-01,  1.07212079e+00],
       [-1.33397722e+01,  2.13684711e+01,  1.39862118e+01,
         7.83744507e+01, -4.04403448e+00,  9.78045368e+00,
        -1.24353323e+01,  2.25498085e+01, -2.60786438e+01,
        -1.32492285e+01, -5.62307739e+01,  1.50671835e+01,
        -1.91685162e+01, -1.47219336e+00,  3.14438343e+01,
        -2.88005581e+01,  6.30527735e+00, -7.56168795e+00,
         1.35009899e+01, -1.85592556e+01, -3.77795258e+

In [ ]:
%%time
threads_per_block = 64
blocks = 24
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)
x_trace, state = Simulator_GPU[blocks, threads_per_block](dt, DeltaT, TotalT, n_sim, theta, rng_states)
# THIS IS FOR THE CUDA GPU

In [ ]:
times_n_sim = np.array([750e-6,2e-3,10e-3,60e-3,650e-3,5.5,55])
total_time_steps_n_sims = np.array([1,10,100,1000,10000,100000,1000000])*1000
times_single_sim = np.array([100e-6,100e-6,600e-6,6e-3,50e-3,350e-3,3.4,35])
total_time_steps_single_sim = np.array([10,100,1000,10000,100000,1000000,10000000,100000000])


plt.scatter(total_time_steps_n_sims, times_n_sim, label = 'Multiple Simulation Time')
plt.scatter(total_time_steps_single_sim, times_single_sim, label = 'Single Simulation Time')
plt.xscale('log')
plt.yscale('log')
plt.title('Time vs Total Time Steps')
plt.xlabel('Total Time Steps')
plt.ylabel('Time (s)')
plt.legend();

In [ ]:
# Provare a chainare diverse simulazione passando i_state